In [ ]:
import pandas as pd
from tqdm.auto import tqdm

In [ ]:
API_KEY = "YOUR API KEY"
USER_AGENT = 'Dataquest'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
artist_tracks_mapping = pd.read_csv('/content/drive/MyDrive/GNN_MT/new_artist_tracks_mapping_df.csv')
artist_tracks_mapping = artist_tracks_mapping.rename(columns = {'Song_Node_ID':'Song_ID'})
df = artist_tracks_mapping

In [ ]:
df.head(), len(df)

(       Artist                                          Song_Name  Song_ID
 0  Black Kids  I'm Not Gonna Teach Your Boyfriend How to Danc...     3307
 1  Black Kids                                Hit The Heartbrakes     3308
 2  Black Kids               I've Underestimated My Charm (Again)     3309
 3  Black Kids                                   Partie Traumatic     3310
 4  Black Kids                             I'm Making Eyes at You     3311,
 252013)

# Batching Dataframe
Split df into 3 batches to limit API calls

In [ ]:
def split_dataframe_in_batches(df, num_batches=3):
    """Splits a DataFrame into a specified number of batches.

    Args:
        df (pandas.DataFrame): The DataFrame to split.
        num_batches (int, optional): The number of batches to split the DataFrame into. Defaults to 5.

    Returns:
        list: A list of DataFrames, each representing a batch.
    """

    if num_batches <= 0:
        raise ValueError("Number of batches must be positive.")

    batch_size = int(len(df) / num_batches)
    batches = [df.iloc[i * batch_size:(i + 1) * batch_size] for i in range(num_batches)]

    # Handle the case where the DataFrame length isn't perfectly divisible by num_batches
    if len(df) % num_batches > 0:
        batches[-1] = pd.concat([batches[-1], df.iloc[num_batches * batch_size:]])

    return batches

In [ ]:
batched_df = split_dataframe_in_batches(df)

In [ ]:
batched_df

[                      Artist  \
 0                 Black Kids   
 1                 Black Kids   
 2                 Black Kids   
 3                 Black Kids   
 4                 Black Kids   
 ...                      ...   
 83999  Donavon Frankenreiter   
 84000  Donavon Frankenreiter   
 84001  Donavon Frankenreiter   
 84002  Donavon Frankenreiter   
 84003  Donavon Frankenreiter   
 
                                                Song_Name  Song_ID  
 0      I'm Not Gonna Teach Your Boyfriend How to Danc...     3307  
 1                                    Hit The Heartbrakes     3308  
 2                   I've Underestimated My Charm (Again)     3309  
 3                                       Partie Traumatic     3310  
 4                                 I'm Making Eyes at You     3311  
 ...                                                  ...      ...  
 83999                                           Our Love    36079  
 84000                                          Bu

# Scrape Function

In [ ]:
import requests
import time
import pandas as pd
from tqdm import tqdm
import pickle
import os

def get_song_tags(df, artist_col='Artist', song_col='Song_Name', rate_limit=5, max_tags=5, max_retries=5):
    """
    Get tags for songs from the Last.fm API with rate limiting, caching, and retry mechanism.

    Args:
        df (pandas.DataFrame): The DataFrame containing the artist and song names.
        artist_col (str, optional): The name of the column containing the artist names.
            Default is 'artist'.
        song_col (str, optional): The name of the column containing the song names.
            Default is 'song_name'.
        rate_limit (int, optional): The maximum number of requests per second.
            Default is 1.
        max_tags (int, optional): The maximum number of tags to fetch for each song.
            Default is 5.
        max_retries (int, optional): The maximum number of retries for failed requests.
            Default is 5.

    Returns:
        pandas.DataFrame: A DataFrame containing the artist, song name, and tags.
    """
    # Define headers and URL
    headers = {'user-agent': USER_AGENT}
    url = 'https://ws.audioscrobbler.com/2.0/'

    # Keep track of the last request time
    last_request_time = 0

    # Create a new DataFrame to store the results
    result_df = pd.DataFrame(columns=['artist', 'song_name', 'tags'])

    # Check if a cached file exists
    cache_file = 'song_tags_cache.pkl'
    if os.path.exists(cache_file):
        result_df = pd.read_pickle(cache_file)

    # progress bar
    progress_bar = tqdm(total=len(df), unit='songs')

    #iterate 
    for _, row in df.iterrows():
        artist = row[artist_col]
        song_name = row[song_col]
        payload = {
            'method': 'track.getInfo',
            'artist': artist,
            'track': song_name,
            'api_key': API_KEY,
            'format': 'json'
        }

        # Retry mechanism
        retries = 0
        while retries < max_retries:
            # Calculate the delay needed to respect the rate limit (minimum 0.25 seconds)
            current_time = time.time()
            #delay = max(0.05, 1 / rate_limit - (current_time - last_request_time))
            delay = 0.05
            time.sleep(delay)
            last_request_time = current_time

            response = requests.get(url, headers=headers, params=payload)

            # Check for rate limiting headers
            if 'X-RateLimit-Remaining' in response.headers and int(response.headers['X-RateLimit-Remaining']) < 10:
                print(f"Rate limit remaining: {response.headers['X-RateLimit-Remaining']}")

            if response.status_code == 200:
                try:
                    data = response.json()
                    if 'track' in data:
                        tags = [tag['name'] for tag in data['track']['toptags']['tag'][:max_tags]]
                        result_df.loc[len(result_df)] = [artist, song_name, ', '.join(tags)]
                    else:
                        result_df.loc[len(result_df)] = [artist, song_name, 'Failed to fetch tags']
                    break

                except JSONDecodeError:
                    print(f"Failed to decode JSON response for {artist} - {song_name}")
                    result_df.loc[len(result_df)] = [artist, song_name, 'Failed to decode  response']
            else:
                retries += 1
                print(f"Request failed with status code {response.status_code}. Retrying... ({retries}/{max_retries})")

        # Save the results to a cache file
        result_df.to_pickle(cache_file)

        # Update the progress bar
        progress_bar.update(1)

    progress_bar.close()

    return result_df

# Batch 1 Scrape

In [ ]:
results_df_0 = get_song_tags(batched_df[0])

100%|██████████| 84004/84004 [6:49:00<00:00,  3.42songs/s]


In [ ]:
#results_df_0.to_csv('/content/drive/MyDrive/GNN_MT/tags_batch_0.csv', index=False)

In [ ]:
results_df_0 = pd.read_csv('/content/drive/MyDrive/GNN_MT/tags_batch_0.csv')

In [ ]:
results_df_0 # careful! First 4 are double because of chache! (hence 84009  instead of 84005 )

,artist,song_name,tags
0,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love..."
1,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy"
2,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative"
3,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie..."
4,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop"
...,...,...,...
84004,Donavon Frankenreiter,Our Love,"acoustic, Surf, summer, chill, rock"
84005,Donavon Frankenreiter,Butterfly,"acoustic, chill, Mellow, folk, singer-songwriter"
84006,Donavon Frankenreiter,Bend In The Road,"acoustic, Mellow, chill, Donavon Frankenreiter..."
84007,Donavon Frankenreiter,Day Dreamer,"acoustic, chill, folk, Mellow, rock"


# Batch 2 Scrape

In [ ]:
results_df_1 = get_song_tags(batched_df[1])

  1%|          | 967/84004 [01:31<2:11:01, 10.56songs/s]

Gang Starr - The Ownerz generated an exception: name 'JSONDecodeError' is not defined


  5%|▍         | 4037/84004 [05:45<2:08:41, 10.36songs/s]

Rakim - Flow Forever generated an exception: name 'JSONDecodeError' is not defined


  6%|▌         | 4942/84004 [06:59<2:16:01,  9.69songs/s]

...

 39%|███▊      | 32424/84004 [46:24<1:28:09,  9.75songs/s]

Katatonia - Sleeper generated an exception: name 'JSONDecodeError' is not defined


 50%|█████     | 42413/84004 [1:01:00<1:06:26, 10.43songs/s]

Diana Ross and the Supremes - You Can't Hurry Love generated an exception: name 'JSONDecodeError' is not defined


100%|██████████| 84004/84004 [2:03:14<00:00, 11.36songs/s]


In [ ]:
results_df_1.to_csv('/content/drive/MyDrive/GNN_MT/tags_batch_1.csv', index=False)

In [ ]:
results_df_1

,artist,song_name,tags
0,Elmore James,I Need You,"blues, Chicago Blues, Old Blues, Classic Blues..."
1,Eli Young Band,Drunk Last Night,"country, great song, 2013, my favorite, modern..."
2,El Perro del Mar,Walk On By - St Etienne Remix,
3,Eddie Cochran,Little Lou,"rockabilly, rockabilly soul, oldies, 50s, rock..."
4,Donovan,Hampstead Incident,"folk, 60s, singer-songwriter, london, classic ..."
...,...,...,...
83989,Comet Gain,Mainlining Mystery,spoken word
83990,Comet Gain,Emotion Pictures,"sexy, John Peel, jangly, ponyhof"
83991,Comet Gain,Brothers Off The Block,family
83992,Comet Gain,If I Had A Soul,indie


# Batch 3 Scrape


In [ ]:
results_df_2 = get_song_tags(batched_df[2])

100%|██████████| 84005/84005 [2:02:48<00:00, 11.40songs/s]


In [ ]:
results_df_2.to_csv('/content/drive/MyDrive/GNN_MT/tags_batch_2.csv', index=False)

In [ ]:
results_df_2

,artist,song_name,tags
0,Bochum Welt,Feelings on a Screen,"downtempo, electro, loved, title track, synthe..."
1,Dengue Fever,Hummingbird,"birds, khmer, aj playlist, alternative, altern..."
2,Hank Williams III,Cocaine Blues,"country, outlaw country, Alt-country, psychobi..."
3,Alien Sex Fiend,Ignore the Machine,"deathrock, post-punk, goth, 80s, batcave"
4,Alec Empire,Suicide,"digital hardcore, breakcore, Drum and bass, dn..."
...,...,...,...
84000,Sleeperstar,I Was Wrong,"The Vampire Diaries, alternative rock, Vampire..."
84001,Jamie Lancaster,Boys Don't Cry,"jazz, 80s, cover, chillout, Tribute"
84002,Anthony Naples,Mad Disrespect,
84003,Irene,Stardust,"swedish, indie pop, twee, got to love these sw..."


# Clear Chache Songs

In [ ]:

# Check if a cached file exists
cache_file = 'song_tags_cache.pkl'
if os.path.exists(cache_file):
    # Clear the cache by deleting the file
    os.remove(cache_file)
    print("Cache cleared successfully!")


Cache cleared successfully!


# Merging Batches

In [ ]:
results_df_0 = pd.read_csv(('/content/drive/MyDrive/GNN_MT/tags_batch_0.csv'))

In [ ]:
results_df_1 = pd.read_csv(('/content/drive/MyDrive/GNN_MT/tags_batch_1.csv'))

In [ ]:
results_df_2 = pd.read_csv(('/content/drive/MyDrive/GNN_MT/tags_batch_2.csv'))

In [ ]:
results_df_0.head(11) #I have the first 5 rows wrongly cached and now have to delete them since their are appearing twice

,artist,song_name,tags
0,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love..."
1,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy"
2,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative"
3,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie..."
4,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop"
5,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love..."
6,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy"
7,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative"
8,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie..."
9,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop"


In [ ]:
# Drop the first 5 rows
results_df_0 = results_df_0.drop(results_df_0.index[:5])

In [ ]:
results_df_0

,artist,song_name,tags
5,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love..."
6,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy"
7,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative"
8,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie..."
9,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop"
...,...,...,...
84004,Donavon Frankenreiter,Our Love,"acoustic, Surf, summer, chill, rock"
84005,Donavon Frankenreiter,Butterfly,"acoustic, chill, Mellow, folk, singer-songwriter"
84006,Donavon Frankenreiter,Bend In The Road,"acoustic, Mellow, chill, Donavon Frankenreiter..."
84007,Donavon Frankenreiter,Day Dreamer,"acoustic, chill, folk, Mellow, rock"


In [ ]:
results_df_1

,artist,song_name,tags
0,Elmore James,I Need You,"blues, Chicago Blues, Old Blues, Classic Blues..."
1,Eli Young Band,Drunk Last Night,"country, great song, 2013, my favorite, modern..."
2,El Perro del Mar,Walk On By - St Etienne Remix,NaN
3,Eddie Cochran,Little Lou,"rockabilly, rockabilly soul, oldies, 50s, rock..."
4,Donovan,Hampstead Incident,"folk, 60s, singer-songwriter, london, classic ..."
...,...,...,...
83989,Comet Gain,Mainlining Mystery,spoken word
83990,Comet Gain,Emotion Pictures,"sexy, John Peel, jangly, ponyhof"
83991,Comet Gain,Brothers Off The Block,family
83992,Comet Gain,If I Had A Soul,indie


In [ ]:
results_df_2

,artist,song_name,tags
0,Bochum Welt,Feelings on a Screen,"downtempo, electro, loved, title track, synthe..."
1,Dengue Fever,Hummingbird,"birds, khmer, aj playlist, alternative, altern..."
2,Hank Williams III,Cocaine Blues,"country, outlaw country, Alt-country, psychobi..."
3,Alien Sex Fiend,Ignore the Machine,"deathrock, post-punk, goth, 80s, batcave"
4,Alec Empire,Suicide,"digital hardcore, breakcore, Drum and bass, dn..."
...,...,...,...
84000,Sleeperstar,I Was Wrong,"The Vampire Diaries, alternative rock, Vampire..."
84001,Jamie Lancaster,Boys Don't Cry,"jazz, 80s, cover, chillout, Tribute"
84002,Anthony Naples,Mad Disrespect,NaN
84003,Irene,Stardust,"swedish, indie pop, twee, got to love these sw..."


In [ ]:
dfs = [results_df_0, results_df_1, results_df_2]

In [ ]:
song_tags_df = pd.concat(dfs, ignore_index=True)

In [ ]:
song_tags_df # missing those 10 songs that got skipped by the exception

,artist,song_name,tags
0,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love..."
1,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy"
2,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative"
3,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie..."
4,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop"
...,...,...,...
251998,Sleeperstar,I Was Wrong,"The Vampire Diaries, alternative rock, Vampire..."
251999,Jamie Lancaster,Boys Don't Cry,"jazz, 80s, cover, chillout, Tribute"
252000,Anthony Naples,Mad Disrespect,NaN
252001,Irene,Stardust,"swedish, indie pop, twee, got to love these sw..."


# Post Processing of Tags df

In [ ]:
song_tags_df = song_tags_df.rename(columns={"artist": "Artist", "song_name": "Song_Name"})

In [ ]:
song_tags_df

,Artist,Song_Name,tags
0,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love..."
1,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy"
2,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative"
3,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie..."
4,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop"
...,...,...,...
251998,Sleeperstar,I Was Wrong,"The Vampire Diaries, alternative rock, Vampire..."
251999,Jamie Lancaster,Boys Don't Cry,"jazz, 80s, cover, chillout, Tribute"
252000,Anthony Naples,Mad Disrespect,NaN
252001,Irene,Stardust,"swedish, indie pop, twee, got to love these sw..."


## Missing Tags


In [ ]:
song_tags_df[song_tags_df["tags"] == "Failed to fetch tags"]

,Artist,Song_Name,tags


In [ ]:
song_tags_df[song_tags_df["tags"] == None]

,Artist,Song_Name,tags


In [ ]:
song_tags_df[song_tags_df["Artist"] == "Anthony Naples"]["tags"]

252000    NaN
Name: tags, dtype: object

In [ ]:
#song_tags_df[song_tags_df["Artist"] == "Gang Starr"] #["tags"]

In [ ]:
song_tags_df[song_tags_df["tags"] == "Failed to decode response"]

,Artist,Song_Name,tags


## Check for NaN

In [ ]:
# Count NaN values in each column
nan_counts = song_tags_df.isna().sum()

# Show rows where there are NaN values in any column
rows_with_nan = song_tags_df[song_tags_df.isna().any(axis=1)]
missing_tags_df = rows_with_nan

print("NaN counts per column:")
print(nan_counts)
print("\nRows with NaN values:")
print(rows_with_nan)

NaN counts per column:
Artist           0
Song_Name        0
tags         18624
dtype: int64

Rows with NaN values:
                Artist              Song_Name tags
12          Black Kids   V-Card (Not Nuthin’)  NaN
13          Black Kids   Natural Born Kissers  NaN
14          Black Kids              In A Song  NaN
15          Black Kids  If My Heart Is Broken  NaN
16          Black Kids       All The Emotions  NaN
...                ...                    ...  ...
251882             Mar            Our Attempt  NaN
251947   Cautious Clay               Cold War  NaN
251952     Xavier Omär              Blind Man  NaN
251961      Iman Omari                 Energy  NaN
252000  Anthony Naples         Mad Disrespect  NaN

[18624 rows x 3 columns]


### Checking if those NaNs really dont have any output
Answer: yes they are indeed empty

In [ ]:
import requests
import time

def scrape_songs(songs):
  # Your Last.fm API key
  API_KEY = 'fe807504fb6ad34b77643919ac43d822'
  USER_AGENT = 'Dataquest'

  # List of songs in the format 'artist - title'
  songs = songs

  # Batch size (adjust according to Last.fm's rate limits)
  BATCH_SIZE = 5

  # Rate limit per second (adjust according to Last.fm's rate limits)
  RATE_LIMIT = 5

  # Keep track of the last request time
  last_request_time = 0

  headers = {
      'user-agent': USER_AGENT
  }

  for i in range(0, len(songs), BATCH_SIZE):
      batch = songs[i:i+BATCH_SIZE]
      for song in batch:
          artist = song[0]
          title = song[1]
          #print(artist,title)
          url = f'http://ws.audioscrobbler.com/2.0/?method=track.getInfo&api_key={API_KEY}&artist={artist}&track={title}&format=json'

          # Calculate the delay needed to respect the rate limit
          current_time = time.time()
          delay = 1 / RATE_LIMIT - (current_time - last_request_time)
          if delay > 0:
              time.sleep(delay)
          last_request_time = time.time()

          response = requests.get(url)
          data = response.json()
          if 'track' in data:
              tags = [tag['name'] for tag in data['track']['toptags']['tag']]
              print(f"Tags for '{song}': {', '.join(tags)}")
          else:
              print(f"Failed to fetch tags for '{song}'")

In [ ]:
#songs = song_tags_df[['Artist','Song_Name']].head(10).values.tolist() # check if the function would return tags if there were any. Answer: Yes.
songs = missing_tags_df[['Artist','Song_Name']].head(10).values.tolist() # check for actual missing tags again: (Yes there are still none)
scrape_songs(songs)

Tags for '['Black Kids', 'V-Card (Not Nuthin’)']': 
Tags for '['Black Kids', 'Natural Born Kissers']': 
Tags for '['Black Kids', 'In A Song']': 
Tags for '['Black Kids', 'If My Heart Is Broken']': 
Tags for '['Black Kids', 'All The Emotions']': 
Tags for '['Black Kids', 'Way Into Leather']': 
Tags for '['Black Kids', "I'm Not Gonna Teach Your Boyfriend How to Dance with You - The Twelves Remix"]': 
Tags for '['The Vaccines', 'Rolling Stones']': 
Tags for '['The Vaccines', 'Maybe (Luck of the Draw)']': 
Tags for '['The Vaccines', 'Take It Easy']': 


## Check 10 Missing Songs
10 songs missing that should have been added with an exception, but they were just skipped. SO I am searching for their tags again. Turns out the actually have tags, so there was some error when calling them the first time. (but that is fine, can happen ofc)

In [ ]:
# Find missing songs
missing_songs_df = pd.merge(artist_tracks_mapping[['Artist', 'Song_Name']], song_tags_df,
                            on=['Artist', 'Song_Name'],
                            how='left',
                            indicator=True)

missing_songs = missing_songs_df[missing_songs_df['_merge'] == 'left_only'][['Artist', 'Song_Name']]

print("Missing songs:")
print(missing_songs)

Missing songs:
                             Artist             Song_Name
84942                    Gang Starr            The Ownerz
88013                         Rakim          Flow Forever
88918                     Nevermore     Inside Four Walls
91488                   Grupo Niche                  Eres
91891            Necessary Response                 Vapor
98176                     Juveniles             Ambitions
112760                     Daedelus   Get Off Your HiHats
114127                      Floetry             Let Me In
116401                    Katatonia               Sleeper
126389  Diana Ross and the Supremes  You Can't Hurry Love


In [ ]:
songs = ['Gang Starr - The Ownerz']
scrape_songs(songs)

Tags for 'Gang Starr - The Ownerz': 


In [ ]:
missing_songs_found = get_song_tags(missing_songs)

100%|██████████| 10/10 [00:02<00:00,  4.52songs/s]


In [ ]:
missing_songs_found

,artist,song_name,tags
0,Gang Starr,The Ownerz,"Hip-Hop, hip hop, rap, Gang Starr, hiphop"
1,Rakim,Flow Forever,"Hip-Hop, hip hop, rap, hiphop, east coast rap"
2,Nevermore,Inside Four Walls,"Progressive metal, thrash metal, Power metal, ..."
3,Grupo Niche,Eres,salsa
4,Necessary Response,Vapor,"electronic, futurepop, synthpop, ebm, industrial"
5,Juveniles,Ambitions,"french, electro, 10s"
6,Daedelus,Get Off Your HiHats,"american i like, electronic, funky, chill, dub..."
7,Floetry,Let Me In,"Neo-Soul, soul, floetry, rnb, smooth"
8,Katatonia,Sleeper,"doom metal, Progressive metal, metal, Progress..."
9,Diana Ross and the Supremes,You Can't Hurry Love,"motown, soul, oldies, 60s, pop"


### Add to tags_df

In [ ]:
missing_songs_found = missing_songs_found.rename(columns={"artist": "Artist", "song_name": "Song_Name"})

In [ ]:
updated_song_tags_df = pd.concat([song_tags_df, missing_songs_found], ignore_index=True)

In [ ]:
updated_song_tags_df

,Artist,Song_Name,tags
0,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love..."
1,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy"
2,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative"
3,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie..."
4,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop"
...,...,...,...
252008,Juveniles,Ambitions,"french, electro, 10s"
252009,Daedelus,Get Off Your HiHats,"american i like, electronic, funky, chill, dub..."
252010,Floetry,Let Me In,"Neo-Soul, soul, floetry, rnb, smooth"
252011,Katatonia,Sleeper,"doom metal, Progressive metal, metal, Progress..."


## Impute Top 3 tags for Empty / NaN Tags
with top 3 Tags of the Artists Songs

In [ ]:
# Extract artists from tracks with missing tags
missing_artists = missing_tags_df['Artist'].unique()

In [ ]:
missing_tags_df.iloc[[1,2,3,4]]

,Artist,Song_Name,tags
13,Black Kids,Natural Born Kissers,NaN
14,Black Kids,In A Song,NaN
15,Black Kids,If My Heart Is Broken,NaN
16,Black Kids,All The Emotions,NaN


In [ ]:
missing_artists

array(['Black Kids', 'The Vaccines', 'Panic! at the Disco', ...,
       'Xavier Omär', 'Iman Omari', 'Anthony Naples'], dtype=object)

In [ ]:
import tqdm
# Compute the 3 most appearing tags for each respective artist
imputed_tags = []

#iterate over artists
for artist in tqdm.tqdm(missing_artists):
    artist_df = updated_song_tags_df[updated_song_tags_df['Artist'] == artist]
    #print(artist)

    artist_tags_list = []

    # Combine tags from all songs by the artist
    for artist_tags in artist_df['tags']:

      artist_tags = str(artist_tags).split(', ')
      #print(artist_tags)
      #print("========")

      for tags in artist_tags:
        #exclude any nan tags
        if tags != "nan":
          artist_tags_list.append(tags)
          #print(tags)

    #print(artist_tags_list)

    # Count the frequency of each tag
    tag_counts = pd.Series(artist_tags_list).value_counts()
    # Get the 3 most common tags
    top_tags = tag_counts.head(3).index.tolist()

    #print(artist)
    #print(top_tags)
    #print("==========")

    # Impute the most common tags to the missing songs by the artist
    for index, row in missing_tags_df[missing_tags_df['Artist'] == artist].iterrows():
        imputed_tags.append({
            'Artist': artist,
            'Song_Name': row['Song_Name'],
            'tags': ', '.join(top_tags)
        })

  0%|          | 5/7135 [00:00<02:55, 40.60it/s]

Black Kids
['indie pop', 'indie', 'black kids']
The Vaccines
['indie rock', 'indie', 'british']
Panic! at the Disco
['rock', 'alternative', 'indie']
The xx
['indie', 'alternative', 'british']
Architecture in Helsinki
['indie', 'indie pop', 'australian']
Of Monsters and Men
['indie', 'icelandic', 'folk']
Grouplove
['indie rock', 'indie pop', 'indie']
Two Door Cinema Club
['indie', 'indie rock', 'electronic']
Slow Club
['indie', 'british', 'folk']


  0%|          | 15/7135 [00:00<02:59, 39.77it/s]

White Lies
['indie rock', 'indie', 'british']
We Have Band
['indie', 'electronic', 'indie rock']
Empire of the Sun
['electronic', 'australian', 'pop']
The Ting Tings
['indie', 'electronic', 'british']
Electric Guest
['indie', 'electronic', 'pop']
The Maccabees
['indie', 'indie rock', 'british']
San Cisco
['indie pop', 'indie', 'australian']
Cold Specks
['fip', 'folk', 'Alternative Folk']


  0%|          | 23/7135 [00:00<03:08, 37.68it/s]

...

Axel
[]
Queen Naija
[]
Maliibu n Helene
[]
Arin Ray
[]
TM404
['downtempo', 'techno']


100%|█████████▉| 7128/7135 [03:45<00:00, 25.41it/s]

Rhian Sheehan
['instrumental', 'electronic', 'Awesome']
Rescate
['rescate vean', 'spanish', 'rock']
Duki
[]
Gabriela Rocha
[]
Guilherme & Santiago
[]


100%|██████████| 7135/7135 [03:46<00:00, 31.55it/s]

Mar
[]
Cautious Clay
[]
Xavier Omär
[]
Iman Omari
[]
Anthony Naples
[]


In [ ]:
# Impute these most common tags to the songs with missing tags
imputed_tags_df = pd.DataFrame(imputed_tags)
imputed_tags_df

,artist,song_name,tags
0,Black Kids,V-Card (Not Nuthin’),"indie pop, indie, black kids"
1,Black Kids,Natural Born Kissers,"indie pop, indie, black kids"
2,Black Kids,In A Song,"indie pop, indie, black kids"
3,Black Kids,If My Heart Is Broken,"indie pop, indie, black kids"
4,Black Kids,All The Emotions,"indie pop, indie, black kids"
...,...,...,...
18619,Mar,Our Attempt,
18620,Cautious Clay,Cold War,
18621,Xavier Omär,Blind Man,
18622,Iman Omari,Energy,


In [ ]:
imputed_tags_df[imputed_tags_df['tags'] != ""]


,artist,song_name,tags
0,Black Kids,V-Card (Not Nuthin’),"indie pop, indie, black kids"
1,Black Kids,Natural Born Kissers,"indie pop, indie, black kids"
2,Black Kids,In A Song,"indie pop, indie, black kids"
3,Black Kids,If My Heart Is Broken,"indie pop, indie, black kids"
4,Black Kids,All The Emotions,"indie pop, indie, black kids"
...,...,...,...
18611,Rhian Sheehan,Standing In Silence Pt 11,"instrumental, electronic, Awesome"
18612,Rescate,La Calle,"rescate vean, spanish, rock"
18613,Rescate,Loco,"rescate vean, spanish, rock"
18614,Rescate,Vuelve a empezar,"rescate vean, spanish, rock"


In [ ]:
imputed_tags_df[imputed_tags_df['tags'] == ""]

,artist,song_name,tags
395,Aláfia,Salve Geral,
396,Aláfia,Corpura,
397,Aláfia,Blacksmith,
398,Aláfia,Preto Cismado,
399,Aláfia,Adinkras,
...,...,...,...
18619,Mar,Our Attempt,
18620,Cautious Clay,Cold War,
18621,Xavier Omär,Blind Man,
18622,Iman Omari,Energy,


### Search for artist tags
and impute them for the ~3k songs that are still missing

In [ ]:
missing_artists_3k = imputed_tags_df[imputed_tags_df['tags'] == ""]["artist"].unique()

In [ ]:
missing_artists_3k

array(['Aláfia', 'Tereza', 'Pikachunes', ..., 'Xavier Omär', 'Iman Omari',
       'Anthony Naples'], dtype=object)

In [ ]:
import requests
import time

def scrape_artist_top_tags(artist_names):
  # Your Last.fm API key
  API_KEY = 'fe807504fb6ad34b77643919ac43d822'
  USER_AGENT = 'Dataquest'

  # Base URL for the API call
  base_url = "http://ws.audioscrobbler.com/2.0/"

  # API method and format
  method = "artist.getTopTags"
  format = "json"


  # Create a new DataFrame to store the results
  top_artists_tags_df = pd.DataFrame(columns=['Artist', 'tags'])
  # list for artists with no tags
  not_tags = []

  # iterate over all artists
  for artist_name in tqdm(artist_names):

    # Parameters for the request
    params = {
        "method": method,
        "artist": artist_name,
        "api_key": API_KEY,
        "format": format,
    }

    headers = {
        "User-Agent": USER_AGENT,
    }

    # Send the request
    response = requests.get(base_url, params=params, headers=headers)
    response.raise_for_status()  # Raise exception for non-200 status codes

    # Extract tags from the response
    data = response.json()
    if "toptags" in data and "tag" in data["toptags"]:
        tags = [tag["name"] for tag in data["toptags"]["tag"]][0:8]
        # append to df
        top_artists_tags_df.loc[len(top_artists_tags_df)] = [artist_name, ', '.join(tags)]

        if [tag["name"] for tag in data["toptags"]["tag"]] == "":
          print("No, Tags found for this artist: ", artist_name)
          not_tags.append(artist_name)
    else:
        print(f"Failed to retrieve tags for artist: {artist_name}")
        return None

  print(top_artists_tags_df)

  return top_artists_tags_df

In [ ]:
artists = missing_artists_3k

In [ ]:
#artists = ["Aláfia", "Tereza"]

In [ ]:
top_artists_tags_df = scrape_artist_top_tags(artists)

  0%|          | 0/2205 [00:00<?, ?it/s]

                     Artist                                               tags
0                    Aláfia  brasil, mpb, brazilian, seen live, vocal, soul...
1                    Tereza  indie, rock, alternative, pop, indie pop, braz...
2                Pikachunes  electronic, seen live, indie, New Zealand, ele...
3           Jordan Mackampa  soul, pop, indie pop, seen live, british, sing...
4     Plutão Já Foi Planeta  brazil, seen live, indie pop, brazilian, natal...
...                     ...                                                ...
2200                    Mar  post-rock, ambient, icelandic, experimental, p...
2201          Cautious Clay  rnb, soul, alternative rnb, electronic, seen l...
2202            Xavier Omär  rnb, soul, pop, electronic, american, christia...
2203             Iman Omari  Neo-Soul, electronic, electronica, Hip-Hop, so...
2204         Anthony Naples  House, deep house, electronic, techno, outside...

[2205 rows x 2 columns]


In [ ]:
top_artists_tags_df

,Artist,tags
0,Aláfia,"brasil, mpb, brazilian, seen live, vocal, soul..."
1,Tereza,"indie, rock, alternative, pop, indie pop, braz..."
2,Pikachunes,"electronic, seen live, indie, New Zealand, ele..."
3,Jordan Mackampa,"soul, pop, indie pop, seen live, british, sing..."
4,Plutão Já Foi Planeta,"brazil, seen live, indie pop, brazilian, natal..."
...,...,...
2200,Mar,"post-rock, ambient, icelandic, experimental, p..."
2201,Cautious Clay,"rnb, soul, alternative rnb, electronic, seen l..."
2202,Xavier Omär,"rnb, soul, pop, electronic, american, christia..."
2203,Iman Omari,"Neo-Soul, electronic, electronica, Hip-Hop, so..."


In [ ]:
top_artists_tags_df[top_artists_tags_df['tags'] == ""]

,Artist,tags
14,Angela Josephine,
17,Aury Moore Band,
37,Kayla Stockert & the Como Brothers,
42,Manuela Mauro,
113,Rebecca Leonard,
...,...,...
2170,Maura Bellucci,
2171,Ryu Ito,
2179,Regina Mira,
2180,Light Bin,


### use these tags for all missing songs

In [ ]:
imputed_tags_df = imputed_tags_df.rename(columns={"artist": "Artist", "song_name": "Song_Name"})

In [ ]:
del merged_df

In [ ]:
# Merge DataFrames based on Artist (outer join to keep all songs)
merged_df = imputed_tags_df.merge(top_artists_tags_df, how='outer', on='Artist')

# Fill missing Tags using values from the right DataFrame
merged_df['Tags'] = merged_df['tags_y'].fillna(merged_df['tags_x'])

# Drop unnecessary columns (optional)
merged_df = merged_df.drop(columns=['tags_x', 'tags_y'])

# Rename columns if needed
merged_df.columns = ['Artist', 'Song_Name', 'tags']



In [ ]:
all_missing_tags = merged_df

In [ ]:
all_missing_tags[all_missing_tags['tags'] == ""]

,Artist,Song_Name,tags
3105,Angela Josephine,Glory Lord!,
3110,Aury Moore Band,Let Me,
3203,Kayla Stockert & the Como Brothers,Don't Go Messing With My Heart,
3224,Manuela Mauro,Resto da sola,
6082,Rebecca Leonard,Incandescence,
...,...,...,...
18582,Ryu Ito,Elderflowers,
18590,Regina Mira,The Winding Paths,
18591,Regina Mira,October,
18592,Light Bin,Moment of Clarity,


## Final Merge

In [ ]:
# Merge imputed_tags_df with updated_song_tags_df where tags are missing
merged_df = pd.merge(updated_song_tags_df, all_missing_tags,
                     on=['Artist', 'Song_Name'],
                     how='left',
                     suffixes=('_original', '_imputed'))

# Update the 'tags_original' with 'tags_imputed' where 'tags_original' is missing
merged_df['tags_original'].fillna(merged_df['tags_imputed'], inplace=True)

# Drop the 'tags_imputed' column
merged_df.drop(columns=['tags_imputed'], inplace=True)

merged_df.columns = ['Artist', 'Song_Name', 'Song_Tags']

In [ ]:
merged_df

,Artist,Song_Name,Song_Tags
0,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love..."
1,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy"
2,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative"
3,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie..."
4,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop"
...,...,...,...
252008,Juveniles,Ambitions,"french, electro, 10s"
252009,Daedelus,Get Off Your HiHats,"american i like, electronic, funky, chill, dub..."
252010,Floetry,Let Me In,"Neo-Soul, soul, floetry, rnb, smooth"
252011,Katatonia,Sleeper,"doom metal, Progressive metal, metal, Progress..."


In [ ]:
merged_df[merged_df['Song_Tags'] == ""]

,Artist,Song_Name,Song_Tags
50555,Angela Josephine,Glory Lord!,
50644,Aury Moore Band,Let Me,
51885,Kayla Stockert & the Como Brothers,Don't Go Messing With My Heart,
52132,Manuela Mauro,Resto da sola,
90088,Rebecca Leonard,Incandescence,
...,...,...,...
251589,Ryu Ito,Elderflowers,
251679,Regina Mira,The Winding Paths,
251680,Light Bin,Moment of Clarity,
251681,Mae Ji-Yoon,Vibrations,


In [ ]:
merged_df[merged_df["Song_Tags"].isna()]

,Artist,Song_Name,Song_Tags


In [ ]:
merged_df[merged_df['Song_Tags'] == "nan"]

,Artist,Song_Name,Song_Tags


In [ ]:
merged_df[merged_df['Song_Tags'] == "NaN"]

,Artist,Song_Name,Song_Tags


In [ ]:
merged_df[merged_df['Song_Tags'] == None]

,Artist,Song_Name,Song_Tags


## Join with Song_IDs

In [ ]:
#imputed_tags_df = imputed_tags_df.rename(columns={"artist": "Artist", "song_name": "Song_Name"})

In [ ]:
# Merge the Song_ID column from original_df to merged_df
song_tags_df_final = pd.merge(merged_df, artist_tracks_mapping[['Artist', 'Song_Name', 'Song_ID']],
                     on=['Artist', 'Song_Name'], how='left')



In [ ]:
song_tags_df_final

,Artist,Song_Name,Song_Tags,Song_ID
0,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love...",3307
1,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy",3308
2,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative",3309
3,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie...",3310
4,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop",3311
...,...,...,...,...
252008,Juveniles,Ambitions,"french, electro, 10s",19818
252009,Daedelus,Get Off Your HiHats,"american i like, electronic, funky, chill, dub...",204002
252010,Floetry,Let Me In,"Neo-Soul, soul, floetry, rnb, smooth",222619
252011,Katatonia,Sleeper,"doom metal, Progressive metal, metal, Progress...",145949


### Save Final Tags DF

In [ ]:
song_tags_df_final.to_csv("/content/drive/MyDrive/GNN_MT/Song_Tags/song_tags_df.csv", index=False)

# Check Top Tags

In [2]:
from pathlib import Path
import pandas as pd
song_tags_df_final = pd.read_csv(Path("data/song_tags/song_tags_df.csv"))



In [3]:
song_tags_df_final

,Artist,Song_Name,Song_Tags,Song_ID
0,Black Kids,I'm Not Gonna Teach Your Boyfriend How to Danc...,"indie, indie pop, indie rock, black kids, love...",3307
1,Black Kids,Hit The Heartbrakes,"indie, indie rock, indie pop, rock, catchy",3308
2,Black Kids,I've Underestimated My Charm (Again),"indie, black kids, indie pop, rock, alternative",3309
3,Black Kids,Partie Traumatic,"indie, indie pop, pop, alternative rock, indie...",3310
4,Black Kids,I'm Making Eyes at You,"indie, indie pop, black kids, 2008, pop",3311
...,...,...,...,...
252008,Juveniles,Ambitions,"french, electro, 10s",19818
252009,Daedelus,Get Off Your HiHats,"american i like, electronic, funky, chill, dub...",204002
252010,Floetry,Let Me In,"Neo-Soul, soul, floetry, rnb, smooth",222619
252011,Katatonia,Sleeper,"doom metal, Progressive metal, metal, Progress...",145949


In [29]:
empty_tags = song_tags_df_final[song_tags_df_final['Song_Tags'].isnull()]
empty_tags


,Artist,Song_Name,Song_Tags,Song_ID
50555,Angela Josephine,Glory Lord!,NaN,11432
50644,Aury Moore Band,Let Me,NaN,11445
51885,Kayla Stockert & the Como Brothers,Don't Go Messing With My Heart,NaN,11634
52132,Manuela Mauro,Resto da sola,NaN,11676
90088,Rebecca Leonard,Incandescence,NaN,17729
...,...,...,...,...
251589,Ryu Ito,Elderflowers,NaN,247541
251679,Regina Mira,The Winding Paths,NaN,249072
251680,Light Bin,Moment of Clarity,NaN,249086
251681,Mae Ji-Yoon,Vibrations,NaN,249080


In [14]:
song_tags = song_tags_df_final['Song_Tags'].apply(lambda x: str(x).split(', '))


In [15]:
song_tags

0         [indie, indie pop, indie rock, black kids, lov...
1              [indie, indie rock, indie pop, rock, catchy]
2         [indie, black kids, indie pop, rock, alternative]
3         [indie, indie pop, pop, alternative rock, indi...
4                 [indie, indie pop, black kids, 2008, pop]
                                ...                        
252008                               [french, electro, 10s]
252009    [american i like, electronic, funky, chill, du...
252010               [Neo-Soul, soul, floetry, rnb, smooth]
252011    [doom metal, Progressive metal, metal, Progres...
252012                     [motown, soul, oldies, 60s, pop]
Name: Song_Tags, Length: 252013, dtype: object

In [21]:
from collections import Counter
import pandas as pd

flat_tags = [str(tag) for sublist in song_tags for tag in sublist]
tag_counts = Counter(flat_tags)
top_10_tags = tag_counts.most_common(20)


In [22]:
top_10_tags_df = pd.DataFrame(top_10_tags, columns=['Tag', 'Count'])
display(top_10_tags_df)

,Tag,Count
0,rock,55151
1,pop,35404
2,indie,35355
3,electronic,31470
4,alternative,27215
5,female vocalists,23320
6,metal,16675
7,indie rock,14781
8,alternative rock,14748
9,ambient,11680
